In [77]:
import pandas as pd
import numpy as np
from portsort import portsort

import matplotlib.pyplot as plt

from pathlib import Path
from fndata import FnStockData
from pandas.tseries.offsets import MonthEnd
from pandas.tseries.offsets import YearEnd
from tqdm import tqdm

#### 선견편향 제거를 위해서 사이즈 레깅

In [78]:
### 매월 6월 말 기준으로 리벨런싱
factor_df=pd.read_csv('factor.csv')
# factor_df['size_lag1']=factor_df.groupby('Symbol')['size'].shift(1)
rebalancing_period=sorted(list(set(pd.to_datetime(factor_df['date'])+YearEnd(0)+MonthEnd(-6))))
factor_df['date']=pd.to_datetime(factor_df['date'])


In [79]:
factor_df.columns

Index(['date', 'Symbol', 'FnGuide Sector', '거래정지여부', '관리종목여부',
       '기말발행주식수 (보통)(주)', '매출액(천원)', '매출원가(천원)', '보통주자본금(천원)', '수익률 (1개월)(%)',
       '수정계수', '수정주가(원)', '영업이익(천원)', '이연법인세부채(천원)', '이익잉여금(천원)', '이자비용(천원)',
       '자기주식(천원)', '자본잉여금(천원)', '종가(원)', '총자산(천원)', 'size', 'bm', 'op',
       'invit', 'mom', 'devil_hml'],
      dtype='object')

# 백테스트 짜기

#### 1. 모멘텀 팩터를 제외한 다른 팩터들은 매년 6월 말 리벨런싱/ 모멘텀은 매달 말 리벨런싱.
#### 2. 다른 팩터들은 이미 래깅이 되어있지만 사이즈는 안되어 있어서 레깅함.
#### 3. 포트폴리오 구성할 떄도 6월 말의 시총액 가중평균으로 7월부터 이듬 해 6월까지 리턴을 가중평균 해야함.
#### 4. Independent Sorting

In [222]:
class backtest:

    def __init__(self,factor_df,quantile_1,quantile_2,factor_1,factor_2):
        self.factor_df=factor_df
        self.quantile_1=quantile_1
        self.quantile_2=quantile_2
        self.factor_1=factor_1
        self.factor_2=factor_2
        




    def winsorizing(factor_list, q):
        #factor_list=[i+'w' for i in factor_list]
        self.factor_df[factor_list]=self.factor_df.groupby('date')[factor_list].apply(lambda x: x.clip(x.quantile(q, interpolation='lower'), 
                     x.quantile(1-q, interpolation='higher'), axis=0))


    


    def assign_scores(self,x,quantile_list):
        # 각 그룹에 대해 퀀타일을 계산
        result = x.quantile(q=quantile_list)
        score = pd.Series(np.NaN, index=x.index)
        
        for i in range(len(quantile_list)):
            if i == 0:
                score = np.where(x <= result[quantile_list[i]], i + 1, score)
            else:
                score = np.where((x <= result[quantile_list[i]]) & 
                                (x >= result[quantile_list[i-1]]), 
                                i + 1, score)
        
        # 마지막 퀀타일보다 큰 값에 대해 score 할당
        score = np.where(x > result[quantile_list[-1]], len(quantile_list) + 1, score)
        
        return pd.Series(score, index=x.index)


    def lagging(self,df,factor,lagging):
        temp=pd.pivot_table(df,index='date',columns='Symbol',values=factor,dropna=False).sort_index().shift(lagging)
        self.temp=temp.reset_index().melt(id_vars='date', var_name='Symbol', value_name=factor).dropna()

        
    def sorting(self,dependent_sort=True,lagging1=0,lagging2=0):
        self.test=self.factor_df.copy()
        self.test=self.test.loc[self.test['거래정지여부']=='정상']#### 거래되지 않는 종목들 테스트에서 제외
        self.test=self.test.loc[self.test['관리종목여부']=='정상']
        self.test['rtn']=self.test['수익률 (1개월)(%)']/100## 이름 헷갈려서 바꿈
        
        if lagging1!=0:
            self.lagging(df=self.test,factor=self.factor_1,lagging=lagging1)
            self.test.drop(columns=self.factor_1,inplace=True)
            self.test=pd.merge(self.test,self.temp,how='left',on=['date','Symbol'])
    
        if lagging2!=0:
            self.lagging(df=self.test,factor=self.factor_2,lagging=lagging2)
            self.test.drop(columns=self.factor_2,inplace=True)
            self.test=pd.merge(self.test,self.temp,how='left',on=['date','Symbol'])

        #self.test[self.factor_2]=self.test.groupby('Symbol')[self.factor_2].shift(lagging2)
        #self.test[self.factor_1]=self.test.groupby('Symbol')[self.factor_1].shift(lagging1)
        self.lagging(df=self.test,factor='size',lagging=1)
        self.temp.rename(columns={'size':'size_1'},inplace=True)
        self.test=pd.merge(self.test,self.temp,how='left',on=['date','Symbol'])

    

        
        #self.test['size_1']=self.test.groupby("Symbol")['size'].shift(1)##size 래깅


        
        self.test['score']=self.test.groupby('date')[self.factor_1].transform(func=lambda x: self.assign_scores(x,quantile_list=self.quantile_1))
        ###dependent sort
        if dependent_sort:
            self.test['score2']=self.test.groupby('date')[self.factor_2].transform(func=lambda x: self.assign_scores(x,quantile_list=self.quantile_2))
           

        else: ### independent_sort
            self.test['score2']=self.test.groupby(['date','score'])[self.factor_2].transform(func=lambda x: self.assign_scores(x,quantile_list=self.quantile_2))
            




    def run(self,score1,score2,rebalancing_period=None,value_weighted=True):
        
        self.test['indicator']=np.where((self.test['score']==score1) & (self.test['score2']==score2),1,np.nan)
        #self.result=self.test.loc[self.test['indicator']==1]

        rtn_list=[]
        date_list=[]
        universe=[]
        self.universe={}
        test_period=sorted((list(set(self.test['date']))))
        #self.rebalancing_period=rebalancing_period
        
        
        if rebalancing_period==None: ### 데이터 프레임의 주기와 리벨런싱 주기가 같은 경우 위와 같이 하면 됨
            rebalancing_period=test_period

            
        self.rebalancing_period=rebalancing_period

        start=rebalancing_period[0]


        for i ,date in enumerate(test_period):

        


            if (date>start) & (len(universe)!=0):
                cap_df=self.test.loc[self.test['date']==rebalance_date]
                df=self.test.loc[self.test['date']==date]
        
                
                df=df.loc[df['Symbol'].isin(universe)]
                cap_df=df.loc[df['Symbol'].isin(universe)]
                # df=df.loc[~(df['rtn'].isna())]
                # df=df.loc[~(df['size_1'].isna())]
                df['rtn'].fillna(0,inplace=True)
                cap_df['size_1'].fillna(0,inplace=True)

                if value_weighted:
                    
                  
                    rtn_list.append(np.dot( df['rtn'].values,(cap_df['size_1']/np.sum(cap_df['size_1']))))
                
                else:
                    rtn_list.append(np.sum(df['rtn'])/len(df['rtn']))

                date_list.append(date)
                self.universe[date]=universe        
                

            if date in rebalancing_period:
                tmp_universe=self.test.loc[(self.test['date']==date)&(self.test['indicator']==1)]["Symbol"].values
                #rebalance_date=date

                if len(tmp_universe)==0:
                    pass
                else:
                    universe=tmp_universe
                    rebalance_date=date

        self.result=pd.DataFrame(rtn_list,index=date_list,columns=['port_rtn'])

        return self.result



    def analysis(self):
        pass
        # self.turn_over={}
        # for i in range(1,len(self.universe)):
        #     sub=len([x for x in self.universe[list(self.universe.keys())[i]] if x not in self.universe[list(self.universe.keys())[i-1]]])
        #     self.turn_over[list(self.universe.keys())[i]]=sub/len(self.universe[list(self.universe.keys())[i-1]])

        # self.turn_over=pd.DataFrame(self.turn_over,index=self.turn_over.keys())
        # print(f"turnover: { self.turn_over.loc[self.turn_over>0]}")
        # self.rtn_yearly=((1+self.result).cumprod()-1)**( 12/(len(self.result.dropna())) )
        # self.std_yearly=(self.result.std())*np.sqrt(12)

        # print(f'연환산 수익률: {self.rtn_yearly }')
        # print(f"샤프 비율:{self.rtn_yearly/self.std_yearly}")
        # self.t_val=self.turn_over.mean()/self.turn_over.std()
        # print(f"t_val: {self.t_val}")


        
        

In [223]:
factor_df_size_ffill=factor_df.copy()
factor_df_size_ffill['size']=factor_df_size_ffill.groupby('Symbol')['size'].ffill()
test=backtest(factor_df=factor_df_size_ffill,quantile_1=quantile_list,quantile_2=quantile_list,factor_1='size',factor_2='bm')
test.sorting(lagging2=6)
mp=test.run(score1=1,score2=1,rebalancing_period=rebalancing_period,value_weighted=True)

In [224]:
mp

,port_rtn
2014-07-31,-0.039487
2014-08-31,0.064176
2014-09-30,0.101493
2014-10-31,-0.039678
2014-11-30,0.001837
...,...
2024-05-31,-0.021317
2024-06-30,-0.069823
2024-07-31,0.070984
2024-08-31,-0.025690


In [225]:
from tqdm import tqdm
quantile_list=[0.2,0.4,0.6,0.8]
test=backtest(factor_df=factor_df_size_ffill,quantile_1=quantile_list,quantile_2=quantile_list,factor_1='size',factor_2='bm')
test.sorting(lagging2=6)
result=pd.DataFrame()
for i in tqdm(range(1, 6)):
    for j in range(1,6):
        tmp=test.run(score1=i,score2=j,rebalancing_period=rebalancing_period,value_weighted=True)
        result[f'size_{i}_bm_{6-j}']=tmp

100%|██████████| 5/5 [00:39<00:00,  7.81s/it]


In [226]:
result

,size_1_bm_5,size_1_bm_4,size_1_bm_3,size_1_bm_2,size_1_bm_1,size_2_bm_5,size_2_bm_4,size_2_bm_3,size_2_bm_2,size_2_bm_1,...,size_4_bm_5,size_4_bm_4,size_4_bm_3,size_4_bm_2,size_4_bm_1,size_5_bm_5,size_5_bm_4,size_5_bm_3,size_5_bm_2,size_5_bm_1
2014-07-31,-0.039487,0.042091,0.020378,0.034717,0.050587,0.033714,0.009603,0.001835,0.038068,0.047623,...,-0.019535,0.021047,-0.003936,0.012281,0.054913,0.011534,-0.009102,0.027559,0.080867,0.033110
2014-08-31,0.064176,0.032111,0.040797,-0.026170,0.017902,-0.009516,-0.000381,0.015873,-0.000733,0.040229,...,0.027860,0.047267,0.059745,0.042424,0.051188,0.064053,0.004161,0.004095,0.011170,-0.045553
2014-09-30,0.101493,0.035402,0.052014,0.036268,0.008927,0.028792,0.000610,0.006573,0.004593,-0.015869,...,0.034570,0.018006,0.023370,0.031908,0.025504,0.004381,-0.017902,-0.076786,0.021929,-0.011316
2014-10-31,-0.039678,0.005484,-0.059311,-0.038347,-0.000641,-0.020362,-0.019927,-0.003886,-0.041374,-0.003377,...,-0.023120,-0.007673,0.001124,-0.019758,-0.023114,-0.021451,-0.060859,-0.065274,-0.063176,0.019174
2014-11-30,0.001837,-0.048131,-0.012337,-0.034660,-0.014994,-0.015955,0.060634,-0.016924,-0.016613,0.010481,...,-0.039678,0.009096,-0.031030,-0.019154,-0.032183,-0.037896,0.018357,0.015330,-0.010772,0.125014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-31,-0.021317,0.036306,0.024674,0.017421,0.033855,0.022391,-0.009422,-0.002518,-0.003219,0.025029,...,-0.034502,0.027995,0.013867,0.023535,-0.008878,-0.075500,0.021686,-0.043852,0.038922,-0.024656
2024-06-30,-0.069823,-0.057307,-0.007169,-0.026813,-0.036190,0.006544,-0.079199,-0.013028,-0.002509,0.000118,...,-0.043675,-0.022578,0.029002,0.043564,0.017392,-0.031152,0.022694,0.028594,0.099280,0.034849
2024-07-31,0.070984,0.012166,-0.020234,-0.008389,-0.017494,0.028937,-0.034889,-0.029745,-0.031277,-0.021066,...,-0.071514,-0.076572,-0.066448,-0.040174,-0.033139,-0.003399,-0.057178,0.009359,-0.054606,-0.030063
2024-08-31,-0.025690,-0.051366,-0.059757,-0.021029,-0.001067,-0.041142,-0.051602,-0.051972,-0.051404,-0.026889,...,-0.033293,-0.039685,-0.061877,-0.039356,-0.014747,0.019478,-0.040368,-0.056458,-0.032804,-0.007959


In [227]:
import plotly.express as px
px.line((1+result).cumprod())

/opt/anaconda3/envs/work/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [228]:
result=result*100

In [229]:
result.to_csv('5x5_table.csv')

## 팩터 수익률

In [124]:
factor_df.columns

Index(['date', 'Symbol', 'FnGuide Sector', '거래정지여부', '관리종목여부',
       '기말발행주식수 (보통)(주)', '매출액(천원)', '매출원가(천원)', '보통주자본금(천원)', '수익률 (1개월)(%)',
       '수정계수', '수정주가(원)', '영업이익(천원)', '이연법인세부채(천원)', '이익잉여금(천원)', '이자비용(천원)',
       '자기주식(천원)', '자본잉여금(천원)', '종가(원)', '총자산(천원)', 'size', 'bm', 'op',
       'invit', 'mom', 'devil_hml'],
      dtype='object')

In [230]:
factor_df_size_ffill['devil_hml_m']=factor_df_size_ffill['devil_hml'].copy()

In [232]:
factor_df_size_ffill['devil_hml_m']=factor_df_size_ffill['devil_hml'].copy() ### 매달 리벨런싱하는 것도 만듬
factors=['bm','op','invit','devil_hml','mom','devil_hml_m']
quantile_list1=[0.5]
quantile_list2=[1/3, 1-(1/3)]
factor_result=pd.DataFrame()
quantile_list3=[0.3,0.7]
for factor in tqdm(factors):

        
    for i in range(1,3):
        for j in range(1,4):
            factor_test=backtest(factor_df=factor_df_size_ffill,quantile_1=quantile_list1,quantile_2=quantile_list2,factor_1='size',factor_2=factor)
            factor_test.sorting(lagging2=6)
            if factor=='mom':
                factor_test=backtest(factor_df=factor_df_size_ffill,quantile_1=quantile_list1,quantile_2=quantile_list3,factor_1='size',factor_2=factor)
                factor_test.sorting()
                tmp=factor_test.run(score1=i,score2=j,rebalancing_period=sorted(list(set(factor_df_size_ffill['date']))), value_weighted=True)

            elif factor=='devil_hml_m':
                factor_test=backtest(factor_df=factor_df_size_ffill,quantile_1=quantile_list1,quantile_2=quantile_list2,factor_1='size',factor_2=factor)
                factor_test.sorting()
                tmp=factor_test.run(score1=i,score2=j,rebalancing_period=sorted(list(set(factor_df_size_ffill['date']))), value_weighted=True)

            else:
                tmp=factor_test.run(score1=i,score2=j,rebalancing_period=rebalancing_period, value_weighted=True)
            if j!=2:
                factor_result[f'size_{i}_{factor}_{4-j}']=tmp


  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [03:00<00:00, 30.05s/it]


In [233]:
real_factor_result=pd.DataFrame(index=factor_result.index)

####bm
real_factor_result['HML']=(factor_result['size_1_bm_1']+factor_result['size_2_bm_1']-factor_result['size_1_bm_3']-factor_result['size_2_bm_3'])/2

####op
real_factor_result['RMW']=(factor_result['size_1_op_1']+factor_result['size_2_op_1']-factor_result['size_1_op_3']-factor_result['size_2_op_3'])/2

#####invit
real_factor_result['CMA']= -(factor_result['size_1_invit_1']+factor_result['size_2_invit_1']-factor_result['size_1_invit_3']-factor_result['size_2_invit_3'])/2

#####mom
real_factor_result['UMD']=(factor_result['size_1_mom_1']+factor_result['size_2_mom_1']-factor_result['size_1_mom_3']-factor_result['size_2_mom_3'])/2

#####devil_hml
real_factor_result['devil_HML']=(factor_result['size_1_devil_hml_1']+factor_result['size_2_devil_hml_1']-factor_result['size_1_devil_hml_3']-factor_result['size_2_devil_hml_3'])/2

real_factor_result['devil_HML_m']=(factor_result['size_1_devil_hml_m_1']+factor_result['size_2_devil_hml_m_1']-factor_result['size_1_devil_hml_m_3']-factor_result['size_2_devil_hml_m_3'])/2



In [234]:



# test=factor_df_size_ffill.copy()
# def assign_scores(x,quantile_list):
#         # 각 그룹에 대해 퀀타일을 계산
#         result = x.quantile(q=quantile_list)
#         score = pd.Series(np.NaN, index=x.index)
        
#         for i in range(len(quantile_list)):
#             if i == 0:
#                 score = np.where(x <= result[quantile_list[i]], i + 1, score)
#             else:
#                 score = np.where((x <= result[quantile_list[i]]) & 
#                                 (x >= result[quantile_list[i-1]]), 
#                                 i + 1, score)
        
#         # 마지막 퀀타일보다 큰 값에 대해 score 할당
#         score = np.where(x > result[quantile_list[-1]], len(quantile_list) + 1, score)
        
#         return pd.Series(score, index=x.index)



# def lagging(df,factor,lagging):
#         temp=pd.pivot_table(df,index='date',columns='Symbol',values=factor,dropna=False).sort_index().shift(lagging)
#         temp=temp.reset_index().melt(id_vars='date', var_name='Symbol', value_name=factor).dropna()
#         return temp

# dependent_sort=True

# lagging1=0
# lagging2=6
# factor_1='size'
# factor_2='invit'
# quantile_list1=[0.5]
# quantile_list2=[1/3, 1-(1/3)]
# test=test.loc[test['거래정지여부']=='정상']#### 거래되지 않는 종목들 테스트에서 제외
# test=test.loc[test['관리종목여부']=='정상']
# test['rtn']=test['수익률 (1개월)(%)']/100## 이름 헷갈려서 바꿈

# if lagging1!=0:
#     temp=lagging(df=test,factor=factor_1,lagging=lagging1)
#     test.drop(columns=factor_1,inplace=True)
#     test=pd.merge(test,temp,how='left',on=['date','Symbol'])

# if lagging2!=0:
#     temp=lagging(df=test,factor=factor_2,lagging=lagging2)
#     test.drop(columns=factor_2,inplace=True)
#     test=pd.merge(test,temp,how='left',on=['date','Symbol'])

# #test[factor_2]=test.groupby('Symbol')[factor_2].shift(lagging2)
# #test[factor_1]=test.groupby('Symbol')[factor_1].shift(lagging1)
# temp=lagging(df=test,factor='size',lagging=1)
# temp.rename(columns={'size':'size_1'},inplace=True)
# test=pd.merge(test,temp,how='left',on=['date','Symbol'])

# #test['size_1']=test.groupby("Symbol")['size'].shift(1)##size 래깅



# test['score']=test.groupby('date')[factor_1].transform(func=lambda x: assign_scores(x,quantile_list=quantile_list1))
# ###dependent sort
# if dependent_sort:
#     test['score2']=test.groupby('date')[factor_2].transform(func=lambda x: assign_scores(x,quantile_list=quantile_list2))
    

# else: ### independent_sort
#     test['score2']=test.groupby(['date','score'])[factor_2].transform(func=lambda x: assign_scores(x,quantile_list=quantile_2))

In [33]:
(1+real_factor_result).cumprod()

,HML,RMW,CMA,UMD,devil_HML
2014-07-31,0.962422,1.004022,NaN,0.992057,0.947335
2014-08-31,0.989692,0.995342,NaN,0.941190,0.965754
2014-09-30,0.997093,0.996270,NaN,0.902350,0.980333
2014-10-31,0.982124,0.991890,NaN,0.890528,0.998785
2014-11-30,0.937761,0.973780,NaN,0.980130,0.995657
...,...,...,...,...,...
2024-05-31,0.444950,1.171617,0.799432,0.818769,0.798138
2024-06-30,0.429133,1.149510,0.803719,0.761027,0.768952
2024-07-31,0.433637,1.121443,0.784069,0.753867,0.773482
2024-08-31,0.431162,1.107488,0.801233,0.779735,0.768093


In [34]:
(1+real_factor_result).cumprod()


,HML,RMW,CMA,UMD,devil_HML
2014-07-31,0.962422,1.004022,NaN,0.992057,0.947335
2014-08-31,0.989692,0.995342,NaN,0.941190,0.965754
2014-09-30,0.997093,0.996270,NaN,0.902350,0.980333
2014-10-31,0.982124,0.991890,NaN,0.890528,0.998785
2014-11-30,0.937761,0.973780,NaN,0.980130,0.995657
...,...,...,...,...,...
2024-05-31,0.444950,1.171617,0.799432,0.818769,0.798138
2024-06-30,0.429133,1.149510,0.803719,0.761027,0.768952
2024-07-31,0.433637,1.121443,0.784069,0.753867,0.773482
2024-08-31,0.431162,1.107488,0.801233,0.779735,0.768093


In [240]:
factors=['bm','op','invit']
quantile_list1=[0.5]
quantile_list2=[1/3, 1-(1/3)]
quantile_list3=[0.3,0.7]
factor_result=pd.DataFrame()

for factor in tqdm(factors):

        
    for i in range(1,3):
        for j in range(1,4):
            factor_test=backtest(factor_df=factor_df_size_ffill,quantile_1=quantile_list2,quantile_2=quantile_list1,factor_1=factor,factor_2='size')
            factor_test.sorting(lagging1=6)
            
            if factor=='mom':
                tmp=factor_test.run(score1=j,score2=i,rebalancing_period=sorted(list(set(factor_df_size_ffill['date']))), value_weighted=True)

            else:
                tmp=factor_test.run(score1=j,score2=i,rebalancing_period=rebalancing_period, value_weighted=True)
                #print(f'{i}_{j}')
       
            factor_result[f'size_{i}_{factor}_{4-j}']=tmp


  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [01:06<00:00, 22.30s/it]


In [242]:
factor_result

,size_1_bm_3,size_1_bm_2,size_1_bm_1,size_2_bm_3,size_2_bm_2,size_2_bm_1,size_1_op_3,size_1_op_2,size_1_op_1,size_2_op_3,size_2_op_2,size_2_op_1,size_1_invit_3,size_1_invit_2,size_1_invit_1,size_2_invit_3,size_2_invit_2,size_2_invit_1
2014-07-31,0.003047,0.017739,0.033429,0.000899,0.021369,0.046000,0.024998,0.019238,0.005677,0.011831,0.027116,0.023122,NaN,NaN,NaN,NaN,NaN,NaN
2014-08-31,0.022172,0.025466,0.021465,0.025638,0.012289,-0.029893,0.022011,0.017971,0.038219,-0.002261,0.016122,-0.001589,NaN,NaN,NaN,NaN,NaN,NaN
2014-09-30,0.021684,0.006690,0.006403,0.001025,-0.050292,0.001561,0.024732,-0.002269,0.003288,-0.047731,0.018933,-0.028112,NaN,NaN,NaN,NaN,NaN,NaN
2014-10-31,-0.017110,-0.027916,-0.026990,-0.036518,-0.065365,0.002901,-0.018272,-0.025915,-0.040496,-0.061269,-0.038372,-0.029821,NaN,NaN,NaN,NaN,NaN,NaN
2014-11-30,-0.013897,-0.013587,-0.010977,-0.002557,0.008638,0.084946,-0.007378,-0.022458,-0.004123,0.010369,0.000916,0.043659,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-31,-0.010917,0.005117,0.013824,-0.048317,-0.016812,-0.005097,0.001891,0.006021,-0.000715,-0.063675,0.025918,-0.027502,-0.000772,0.006914,0.003887,-0.002843,-0.016743,-0.046164
2024-06-30,-0.031318,-0.018731,-0.016530,-0.014679,0.044813,0.042343,-0.036588,-0.024455,0.009188,0.031017,0.032331,0.022976,-0.037061,-0.021339,-0.002646,0.007431,0.052091,-0.018730
2024-07-31,0.000680,-0.017725,-0.023517,-0.034304,-0.005379,-0.036953,-0.003663,-0.019478,-0.033652,-0.085978,-0.010667,-0.010587,-0.006607,-0.015042,-0.030485,-0.040165,0.017690,-0.066021
2024-08-31,-0.044668,-0.054627,-0.026906,-0.011105,-0.048375,-0.016705,-0.048573,-0.038735,-0.025202,-0.031427,-0.015977,-0.030569,-0.045715,-0.032013,-0.037190,-0.027983,-0.051670,0.003023


In [243]:
real_factor_result['SMB']=np.sum((factor_result.iloc[:,:3].values-factor_result.iloc[:,4:7].values)/3 + (factor_result.iloc[:,7:10].values-factor_result.iloc[:,10:13].values)/3+(factor_result.iloc[:,13:16].values-factor_result.iloc[:,16:-1].values)/3,axis=1)/3

In [ ]:
#real_factor_result.drop(columns='devil_HML',inplace=True)

In [244]:

from pathlib import Path
from fndata import FnStockData,FnMarketData
from pandas.tseries.offsets import MonthEnd
from pandas.tseries.offsets import YearEnd
CWD = Path('.').resolve()
DATA_DIR = CWD / 'data'
fndata_path = DATA_DIR / '고금계과제_시장수익률_201301-202408.csv'
fn = FnMarketData(fndata_path)
df = fn.get_data(format='wide')

In [245]:
rf_path = DATA_DIR / '통안채1년물_월평균_201301-202408.csv'
rf = pd.read_csv(rf_path)


In [246]:
df=df.reset_index()
rf['date']=df['date']


In [247]:
(rf['원자료']/12)/100

0      0.002267
1      0.002242
2      0.002175
3      0.002150
4      0.002150
         ...   
135    0.002824
136    0.002822
137    0.002793
138    0.002675
139    0.002540
Name: 원자료, Length: 140, dtype: float64

In [248]:
real_factor_result=real_factor_result.reset_index()
real_factor_result.rename(columns={'index':'date'},inplace=True)
real_factor_result

,date,HML,RMW,CMA,UMD,devil_HML,devil_HML_m,SMB
0,2014-07-31,0.037742,-0.004015,NaN,0.007713,0.052827,0.048408,NaN
1,2014-08-31,-0.028119,0.008441,NaN,0.050716,-0.019231,-0.014143,NaN
2,2014-09-30,-0.007372,-0.000913,NaN,0.041267,-0.014955,-0.013480,NaN
3,2014-10-31,0.014770,0.004612,NaN,0.013101,-0.019031,-0.031257,NaN
4,2014-11-30,0.045211,0.018273,NaN,-0.100616,0.003172,0.039507,NaN
...,...,...,...,...,...,...,...,...
118,2024-05-31,0.033980,0.016784,0.019331,0.027295,0.038024,-0.230500,0.003357
119,2024-06-30,0.035905,0.018868,-0.004127,0.072369,0.036968,0.030600,-0.032497
120,2024-07-31,-0.013422,0.022701,0.024867,0.011700,-0.007334,-0.040961,-0.027175
121,2024-08-31,0.006081,0.012114,-0.019766,-0.032099,0.007514,0.003231,0.004686


In [249]:
df['RF']=(rf['원자료']/12)/100
df['Mkt-RF']=df['MKF2000']-df['RF']
real_factor_result=pd.merge(real_factor_result,df[['date','RF','Mkt-RF']],how='left',on=['date'])

In [250]:
real_factor_result[['HML','RMW',"CMA","UMD",'SMB',"RF","Mkt-RF",'devil_HML','devil_HML_m']]=real_factor_result[['HML','RMW',"CMA","UMD",'SMB',"RF","Mkt-RF",'devil_HML','devil_HML_m']]*100

In [251]:
real_factor_result.to_csv('factor_port.csv')

,date,HML,RMW,CMA,UMD,devil_HML,devil_HML_m,SMB,RF,Mkt-RF
0,2014-07-31,-3.757792,0.402188,NaN,-0.794311,-5.266451,-4.840799,NaN,0.207750,2.912250
1,2014-08-31,2.833426,-0.864528,NaN,-5.127432,1.944222,1.377857,NaN,0.198333,-0.188333
2,2014-09-30,0.747814,0.093286,NaN,-4.126683,1.509630,1.340082,NaN,0.190000,-2.870000
3,2014-10-31,-1.501234,-0.439627,NaN,-1.310085,1.882165,3.125667,NaN,0.177917,-2.657917
4,2014-11-30,-4.517046,-1.825836,NaN,10.061575,-0.313124,-3.955391,NaN,0.170583,2.829417
...,...,...,...,...,...,...,...,...,...,...
118,2024-05-31,-3.440904,-1.697048,-1.972677,-2.275630,-3.849619,31.288653,-0.012616,0.282250,-2.282250
119,2024-06-30,-3.554767,-1.886910,0.536301,-7.052341,-3.656764,-6.519602,0.018125,0.279333,2.430667
120,2024-07-31,1.049634,-2.441631,-2.444920,-0.940813,0.589132,4.290000,0.012175,0.267500,-1.817500
121,2024-08-31,-0.570747,-1.244353,2.189145,3.431331,-0.696685,-5.018111,0.003924,0.254000,-3.834000


In [ ]:
draw.dropna()

,HML,RMW,CMA,UMD,devil_HML,devil_HML_m,SMB,RF,Mkt-RF
date,,,,,,,,,
2015-07-31,3.648289,-0.111403,-1.061261,-3.884785,1.600087,2.780931,-2.776863,0.133083,-2.973083
2015-08-31,2.302412,-0.298066,1.208264,1.139650,0.241229,-1.944775,1.582378,0.132000,-4.922000
2015-09-30,0.507126,4.151249,-1.297064,4.970660,-0.734679,-2.776625,-1.340916,0.129583,1.260417
2015-10-31,-1.835570,4.297896,-1.592074,1.112766,-0.877648,-3.982755,1.292937,0.129083,4.000917
2015-11-30,2.294378,3.379682,0.676783,-3.927111,0.990755,2.143806,0.079919,0.136417,-1.666417
...,...,...,...,...,...,...,...,...,...
2024-04-30,-2.363251,-0.232813,-0.907765,-0.837835,-2.809712,-5.885743,-1.325648,0.282417,-2.892417
2024-05-31,-3.440904,-1.697048,-1.972677,-2.275630,-3.849619,31.288653,-1.261643,0.282250,-2.282250
2024-06-30,-3.554767,-1.886910,0.536301,-7.052341,-3.656764,-6.519602,1.812511,0.279333,2.430667


In [211]:
real_factor_result

,date,HML,RMW,CMA,UMD,devil_HML,devil_HML_m,RF,Mkt-RF,SMB
0,2014-07-31,NaN,NaN,NaN,NaN,NaN,NaN,0.207750,2.912250,NaN
1,2014-08-31,NaN,NaN,NaN,NaN,NaN,NaN,0.198333,-0.188333,NaN
2,2014-09-30,-1.501234,-0.439627,NaN,-1.812457,1.882165,2.359091,0.190000,-2.870000,NaN
3,2014-10-31,NaN,NaN,NaN,NaN,-0.313124,NaN,0.177917,-2.657917,NaN
4,2014-11-30,NaN,NaN,NaN,NaN,NaN,NaN,0.170583,2.829417,NaN
...,...,...,...,...,...,...,...,...,...,...
118,2024-05-31,NaN,NaN,NaN,NaN,NaN,10.477013,0.282250,-2.282250,NaN
119,2024-06-30,NaN,NaN,NaN,NaN,NaN,7.110345,0.279333,2.430667,NaN
120,2024-07-31,NaN,NaN,NaN,NaN,NaN,-4.615000,0.267500,-1.817500,NaN
121,2024-08-31,NaN,NaN,NaN,NaN,NaN,-1.488994,0.254000,-3.834000,NaN


In [253]:
draw=real_factor_result.set_index(['date'])
px.line((1+draw.dropna()/100).cumprod())

/opt/anaconda3/envs/work/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [271]:
fn=pd.read_csv('fn_factor.csv')
fn=fn.iloc[7:].T
fn=fn.iloc[6:]
fn.columns=['date','HML_fn',"SMB_fn"]
fn['date']=pd.to_datetime(fn['date'])

In [280]:
fn["HML_fn"].astype(float)/100

Unnamed: 6      0.0409
Unnamed: 7     -0.0175
Unnamed: 8      0.0090
Unnamed: 9      0.0218
Unnamed: 10    -0.0134
                 ...  
Unnamed: 143    0.0620
Unnamed: 144    0.0194
Unnamed: 145   -0.0031
Unnamed: 146   -0.0279
Unnamed: 147   -0.0394
Name: HML_fn, Length: 142, dtype: float64

In [283]:
temp=pd.merge(draw.reset_index(),fn,on=['date'],how='left')
#temp[['HML_fn','SMB_fn']]=temp[['HML_fn','SMB_fn']]
#temp=temp.astype(float)

temp=temp.set_index(['date'])
temp=temp.astype(float)
px.line((1+temp.dropna()/100).cumprod())

/opt/anaconda3/envs/work/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



,date,HML,RMW,CMA,UMD,devil_HML,devil_HML_m,SMB,RF,Mkt-RF,HML_fn,SMB_fn
0,2014-07-31,3.774200,-0.401486,NaN,0.771260,5.282676,4.840799,NaN,0.207750,2.912250,5.80,-1.98
1,2014-08-31,-2.811937,0.844063,NaN,5.071568,-1.923051,-1.414268,NaN,0.198333,-0.188333,2.30,2.31
2,2014-09-30,-0.737216,-0.091258,NaN,4.126683,-1.495506,-1.348040,NaN,0.190000,-2.870000,-2.19,3.91
3,2014-10-31,1.476964,0.461169,NaN,1.310085,-1.903057,-3.125667,NaN,0.177917,-2.657917,-1.69,0.73
4,2014-11-30,4.521119,1.827259,NaN,-10.061575,0.317224,3.950715,NaN,0.170583,2.829417,0.36,-3.62
...,...,...,...,...,...,...,...,...,...,...,...,...
118,2024-05-31,3.398030,1.678372,1.933062,2.729514,3.802372,-23.049964,0.335745,0.282250,-2.282250,4.20,2.93
119,2024-06-30,3.590530,1.886776,-0.412697,7.236936,3.696792,3.060000,-3.249664,0.279333,2.430667,6.20,-3.85
120,2024-07-31,-1.342243,2.270111,2.486736,1.170041,-0.733371,-4.096083,-2.717503,0.267500,-1.817500,1.94,-2.65
121,2024-08-31,0.608096,1.211414,-1.976559,-3.209857,0.751437,0.323111,0.468607,0.254000,-3.834000,-0.31,-2.92
